# Саммари

В статье написано, как можно автоматизировать объяснение кластеров. Без линейных комбинаций, без нечетких формулировок - только понятный набор условий и все. Результат приведен на рисунках ниже - названия кластеров выбраны с помощью описанного алгоритма.

![](./images/cluster-vis.png)

# Кластеризация

Кластеризация - очень полезный способ превращения данных в инсайты. Она позволяет объединить те наблюдения о вашем продукте, которые ранее выглядели хаотичным набором точек. 

Вот только стоит вспомнить, что выходом модели кластеризации по определению является соответствие наблюдений и ближайших к ним скоплений - вряд ли вы пойдете с этой кипой чисел в отдел маркетинга. 

Конечно же, сразу приходит на ум множество способов интерпретации кластеров, но к наиболее популярным из них есть вопросы:
- Иногда достаточно знать центры скоплений, которые легко посчитать. Но вряд ли ваш заказчик захочет проводить кампанию среди клиентов, сделавших примерно 23 заказа за последний месяц. А "примерно 23" - это от 22 до 24, или от 10 до 30? 
- Иногда можно применить техники снижения размерности - например, PCA. Но я неплохо так напрягся, объясняя на пальцах одному щепетильному заказчику выражение "2 * log(дни активности - 10) - 3.2 * log(минуты игры)". А ведь это самое обыкновенное выражение для значений, которые попадают на красивую финальную визуализацию

Наверное, можно всего-то найти несколько простых правил, например: "99% пользователей кластера  купили более 40 шмубликов". И потом для кластеров, которые подчиняются правилу, можно было бы придумать короткое описание - например "фанаты шмубликов". Изи-пизи!

Как же это сделать?

# Алгоритм

Для начала давай определим граф сравнений $G = (V, E)$. Это ориентированный граф, все его вершины будут соответствовать кластерам. Каждой дуге графа назначим еще четыре дополнительных атрибута - признак $f$, доля кластера $p$ и пороги отсечения $u, l$. Два кластера будет связывать дуга в том случае, если более $(1 - p)$% значений признака f в первом кластере ниже порога $u$, и такая же часть значений этого же признака во втором кластере выше порога $l$; при этом $u < l$. При наличии нескольких возможных дуг между кластерами выбирается дуга с наименьшим $p$.

Мы можем теперь объединять дуги графа $G$ в множества $S_i$. Каждому множеству $S_i$ соответствует $S'_i$ - множество неориентированных ребер, соединяющих вершины $V$ в случае, если между ними есть хотя бы одна дуга в $S_i$. Кроме того, некоторым $S_i$ можно сопоставить простое правило, которое далее станет частью интерпретации - например, в следующем виде:

$$((c_1, c_3), (c_2, c_4), daily\_inapp\_time, 9\ min, 1\%)$$

Такие правила достаточно просто объяснять. Например, для примера выше можно назвать кластеры $c2, с4$ залипшими игроками и знать, что большая часть этих кластеров в среднем играет от 9 минут в день. 

Если теперь мы найдем покрытие множества ребер полного графа $K_{|V|}$ множествами $S'_i$ то мы найдем набор правил, который отличит каждый кластер от всех остальных. Это и будет искомой интерпретацией!

Однако как выбрать $S_i$, чтобы по нему можно было составить правило? Можно предложить следующий способ его формирования:
1. Проверить, все ли дуги в $S_i$ связаны с одним и тем же признаком. Если нет - правила не существует.
2. Проверить, нет ли в $S_i$ пар дуг, в которой кластер $c$ являлся бы началом одной дуги и концом другой. Если такой кластер есть - правила не существует.
3. Проверить, существует ли точка, которая существует на отрезках $(u, l)$ для всех дуг множества. Если такая точка  есть - выбрать ее в качестве порога отсечения правила. Если такой точки нет - правила не существует.
4. Выбрать максимальное $p$ среди всех дуг в качестве $p$ правила 

Таким образом, алгоритм интерпретации выглядит так:
1. Формируем граф сравнений $G$
2. Находим все $S_i$, для которых существует правило в заданном виде
3. Ищем покрытие ребер $K_{|V|}$ множествами $S'_i$

Я не буду пока приводить здесь детали реализации алгоритма - он громоздкий, требует доработки и на данный момент подходит лишь для небольшого количества кластеров и признаков. Тем не менее, его уже можно попробовать в деле. 

# Практика

У меня под рукой чисто случайно оказался старый датасет сервиса menu.by. В нем я нашел информацию о пользователях и их заказах. Я аггрегировал предпочтения клиентов и получил следующую таблицу

![](./images/features.png)

Выше найдете следующие признаки:
- active_days - количество дней активности
- avg_order - средний чек пользователя, выраженный в копейках BYN
- items_per_order - среднее количество предметов на один заказ
- unique_items_per_order - среднее количество уникальных пунктов на один заказ
- lifetime - время жизни пользователя сервиса

Все признаки (кроме lifetime) аггрегированы за последний месяц. Давайте посмотрим на их распределения. Разумеется, сперва логарифмируем значения некоторых признаков - иначе их распределения будут иметь длинный хвост, и мы ничего путного не увидим:

![](./images/distributions.png)

Вы слышите отдаленные крики? Это столбец lifetime = 0 требует, чтобы для него назначили отдельный признак. Добавляем в таблицу поле one_day_user - для случая, когда пользователь был активен только один день. 

Поскольку я исчерпал дневной лимит на препроцессинг признаков, начинаем кластеризовать. Используем православный k-means, чтобы получить долю ненависти со стороны DS-сообщества, и выделяем кластеры. После этого настает время для нашего супер-пупер-дупер объяснителя! Запускаем его для всех найденных кластеров и признаков датасета - стараемся найти интерпретацию, которая будет охватывать хотя бы 60% кластера (порог можно менять).

```python
from explanation import ClustersExplanation

explanation = ClustersExplanation(
    clusters, 
    features,
    thresholds=np.linspace(0, 0.4, 26)
)

explanation.fit(users_df)

explanation.score()
# 1.0 - значит, все пары кластеров различаются хотя бы одним правилом

pd.DataFrame(explanation.explain())
```
![](./images/rules.png)

В таблице выше приведены правила, по которым кластеры можно отличить друг от друга. Для того, чтобы назвать правила, придется применить фантазию. Фрагмент кода ниже позволяет получить названия кластеров и интерпретацию каждого из правил в таблице

```python

rule_interpretation = [
    ('inactive', ''), # Пользователи в приложении меньше 1-го дня точно не являются активными
    ('low-spending', ''), # 19 BYN на момент сбора данных были равны почти 10 USD - не очень большая трата
    ('', 'mass-buying'), # Заказ, который содержит от 4-х предметов, можно назвать большим
    ('single-minded', ''), # Часто ли вы составляете заказ из одного или двух пунктов? Я - редко. Может, этим ребятам не предложили имбирь и палочки
    ('fresh', 'experienced'), # Пользователи, которые в нашем сервисе уже более полугода, а также новые клиенты
    ('returning', 'first-time') # Сегодняшняя порция клиентов, а также те, кто вернулся сюда снова
]

explanation.get_legend(rule_interpretation)
```
![](./images/rule-names.png)

```python
explanation.get_cluster_names(rule_interpretation)
```
![](./images/cluster-names.png)

```python 
sns.scatterplot(
    data=users_pca_df,
    x="x",
    y="y",
    hue="cluster_label"
)
```
![](./images/cluster-vis.png)
Такая визуализация может стать готовым набором действий для проведения кампании. Например, может навести на мысль о том, чтобы предложить кластеру возвращающихся новичков с ограниченными предпочтениями товар-якорь со скидкой для увеличения их лояльности. 